In [1]:
import pandas as pd
import sqlite3

### Creating SQLite Database

In [2]:
con = sqlite3.connect('library_db.db')
curs = con.cursor()

In [3]:
sql_file = open('sqlite.sql', "r")

In [4]:
def execute_commands(sql_file, sep, database):
    '''
    This function execute all commands of a .sql file.
    It separates the commands using a separator set
    by the user
    '''
    try:
        con = sqlite3.connect(f'{database}')
        curs = con.cursor()
        sql_file = open(f'{sql_file}', "r")
        content_sql = sql_file.read().split(f'{sep}')
        for command in content_sql:
            curs.execute(command)
        print('All commands were succesfully executed.')
    except:
        print('There is something wrong. \nCheck parameters then try again.')

In [5]:
execute_commands(sql_file='sqlite.sql', sep='\n\n', database='library_db.db')

All commands were succesfully executed.


### Dataset

In [6]:
df = pd.read_csv('books.csv', error_bad_lines=False)

b'Skipping line 3350: expected 12 fields, saw 13\nSkipping line 4704: expected 12 fields, saw 13\nSkipping line 5879: expected 12 fields, saw 13\nSkipping line 8981: expected 12 fields, saw 13\n'


In [7]:
df.columns

Index(['bookID', 'title', 'authors', 'average_rating', 'isbn', 'isbn13',
       'language_code', '  num_pages', 'ratings_count', 'text_reviews_count',
       'publication_date', 'publisher'],
      dtype='object')

### Function - Applying First Normal Form

In [8]:
def apply_fist_nf(df:pd.DataFrame, feature:str, sep:str):
    '''
    This function transform occurences with more than one value like 'authors: author_1/author_2' into
    occurences with an unique value 'author_1', 'author_2', ensuring first normal form
    '''
    new_df = pd.DataFrame([feature])
    new_df.loc[0, feature] = 'init'
    for i in range(df.shape[0]):
        occurences = df.loc[i, feature].split(sep)
        for occurence in occurences:
            if (occurence not in new_df[feature].unique()):
                new_df.loc[new_df.shape[0], feature] = occurence
    new_df.drop(0, axis=1, inplace=True)
    new_df.drop(0, axis=0, inplace=True)
    new_df.reset_index(inplace=True)
    new_df.drop('index', axis=1, inplace=True)
    return new_df


### Publishers

In [9]:
publisher_df = pd.DataFrame(df['publisher'].unique())
publisher_df.rename(columns={0:'publisher'}, inplace=True)

In [10]:
publisher_df.drop(0, axis=0, inplace=True)
publisher_df.reset_index(inplace=True)
publisher_df.drop('index', axis=1, inplace=True)

In [11]:
publisher_df.head()

,publisher
0,Scholastic
1,Nimble Books
2,Gramercy Books
3,Del Rey Books
4,Crown


In [12]:
for index, row in publisher_df.iterrows():
    curs.execute('INSERT INTO publisher (name) VALUES (?)',[row.publisher])
con.commit()

### Author

In [13]:
author_df = pd.DataFrame(df['authors'].unique())
author_df.rename(columns={0:'author'}, inplace=True)

In [14]:
author_df.head()

,author
0,J.K. Rowling/Mary GrandPré
1,J.K. Rowling
2,W. Frederick Zimmerman
3,Douglas Adams
4,Douglas Adams/Stephen Fry


In [15]:
author_df = apply_fist_nf(df=author_df, feature='author', sep='/')

In [16]:
author_df.head()

,author
0,J.K. Rowling
1,Mary GrandPré
2,W. Frederick Zimmerman
3,Douglas Adams
4,Stephen Fry


In [17]:
for index, row in author_df.iterrows():
    curs.execute('INSERT INTO author (name) VALUES (?)',[row.author])
con.commit()

### Book 

#### Treating books_df

In [84]:
book_df = df.copy()
book_df.drop('bookID', axis=1, inplace=True)
book_df.rename(columns={'  num_pages':'pages_number'},inplace=True)
book_df = book_df[['authors','title','pages_number', 'publisher']] 

In [85]:
book_df['publisher'].unique()

array(['Scholastic Inc.', 'Scholastic', 'Nimble Books', ..., 'Suma',
       'Panamericana Editorial', 'Editorial Presença'], dtype=object)

In [86]:
book_df['publisher'] = book_df['publisher'].replace('Scholastic Inc.', 'Scholastic')

In [87]:
book_df.head()

,authors,title,pages_number,publisher
0,J.K. Rowling/Mary GrandPré,Harry Potter and the Half-Blood Prince (Harry ...,652,Scholastic
1,J.K. Rowling/Mary GrandPré,Harry Potter and the Order of the Phoenix (Har...,870,Scholastic
2,J.K. Rowling,Harry Potter and the Chamber of Secrets (Harry...,352,Scholastic
3,J.K. Rowling/Mary GrandPré,Harry Potter and the Prisoner of Azkaban (Harr...,435,Scholastic
4,J.K. Rowling/Mary GrandPré,Harry Potter Boxed Set Books 1-5 (Harry Potte...,2690,Scholastic


In [88]:
book_df['publisher'] = [list(publisher_df.index[publisher_df['publisher'] == book_df.loc[i,'publisher']])\
    [0] for i in range(book_df.shape[0])]

In [89]:
book_df.rename(columns={'publisher':'id_publisher'}, inplace=True)
book_df.head(10)

,authors,title,pages_number,id_publisher
0,J.K. Rowling/Mary GrandPré,Harry Potter and the Half-Blood Prince (Harry ...,652,0
1,J.K. Rowling/Mary GrandPré,Harry Potter and the Order of the Phoenix (Har...,870,0
2,J.K. Rowling,Harry Potter and the Chamber of Secrets (Harry...,352,0
3,J.K. Rowling/Mary GrandPré,Harry Potter and the Prisoner of Azkaban (Harr...,435,0
4,J.K. Rowling/Mary GrandPré,Harry Potter Boxed Set Books 1-5 (Harry Potte...,2690,0
5,W. Frederick Zimmerman,"Unauthorized Harry Potter Book Seven News: ""Ha...",152,1
6,J.K. Rowling,Harry Potter Collection (Harry Potter #1-6),3342,0
7,Douglas Adams,The Ultimate Hitchhiker's Guide: Five Complete...,815,2
8,Douglas Adams,The Ultimate Hitchhiker's Guide to the Galaxy ...,815,3
9,Douglas Adams,The Hitchhiker's Guide to the Galaxy (Hitchhik...,215,4


In [67]:
id_publisher = publisher_df.index[publisher_df['publisher'] == book_df.loc[5,'publisher']]

In [68]:
list(id_publisher)[0]

1

In [57]:
book_df['publisher'].head(10)

0        Scholastic
1        Scholastic
2        Scholastic
3        Scholastic
4        Scholastic
5      Nimble Books
6        Scholastic
7    Gramercy Books
8     Del Rey Books
9             Crown
Name: publisher, dtype: object

### 